In [1]:
from skimage import io
import glob
import os
from natsort import natsorted
import numpy as np

In [3]:
image_fns = glob.glob('/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/*tif*')

In [4]:
os.listdir('/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1')

['Assaylayout', 'Images', 'FFC_Profile']

In [5]:
print(image_fns)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
base_dir = '/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/'
fn_template = os.path.join(base_dir, 'r03c05f01p01-ch*sk1fk1fl1.tiff')
fns = glob.glob(fn_template)

In [7]:
fns = natsorted(fns)
fns

['/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/r03c05f01p01-ch2sk1fk1fl1.tiff',
 '/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/r03c05f01p01-ch3sk1fk1fl1.tiff',
 '/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/r03c05f01p01-ch4sk1fk1fl1.tiff',
 '/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/r03c05f01p01-ch5sk1fk1fl1.tiff',
 '/Volumes/lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan/BPP0050-1-Live-cell-to4i_Fixed_Cy1__2025-04-14T16_57_11-Measurement 1/Images/r03c05f01p01-ch6sk

In [8]:
images = np.stack([io.imread(fn) for fn in fns], axis = 0)

In [15]:
import napari

In [17]:
viewer = napari.Viewer()
viewer.add_image(images, channel_axis=0)

[<Image layer 'Image' at 0x1369ee110>,
 <Image layer 'Image [1]' at 0x315daae90>,
 <Image layer 'Image [2]' at 0x3171c7fa0>,
 <Image layer 'Image [3]' at 0x1369ede70>,
 <Image layer 'Image [4]' at 0x317148c10>]

# Previous code

In [ ]:
base_dir = '/mnt/DATA3/BPP0050'
zarr_dirs = glob.glob(os.path.join(base_dir, '*fixed*/acquisition/zarr/*/'))

In [7]:
for zarr_dir in tqdm(zarr_dirs):
    pos = re.search(r'(\d+)\.zarr', zarr_dir).group(1) ## needs attn
    label_dir = zarr_dir.replace('zarr','labels') ## needs attn
    os.makedirs(label_dir, exist_ok=True)
    
    zarr_group = zarr.open_group(zarr_dir)
    segmentation_input = zarr_group.images[:,1,:,...] ## needs channel enumeration checking
    segmentation_input = np.max(segmentation_input, axis = 1) # check axis enumeration
    segmentation = []
    # for i, segmentation_input_frame in tqdm(enumerate(segmentation_input), total = len(segmentation_input), leave=False):
    masks, flows, styles = model.eval(segmentation_input, #_frame, 
                                      diameter=100, # check the diameter
                                      # anisotropy=1/anisotropy,
                                      # min_size=2000,
                                      # z_axis=0, do_3D=True,
                                     )
        # segmentation.append(masks)
    # segmentation = np.stack(segmentation, axis=0)
    segmentation = masks

    # segmentation_fn = os.path.join(label_dir, 'segmentation.h5')
    # with btrack.io.HDF5FileHandler(segmentation_fn, 'w', obj_type='obj_type_1') as writer:
    #     writer.write_segmentation(segmentation)

    # del segmentation_input

    # CHECK IF WE NEED TO THRESHOLD?
    
    # mtb_images = zarr_group.images[:, 0, ...]
    # mtb_images = np.transpose(mtb_images, (0, 2, 3, 4, 1))
    # mtb_thresh = np.where(mtb_images >= Mtb_load_thresh, True, False)
    # intensity_image = np.stack([mtb_images[...,0], 
    #                             mtb_images[...,1],
    #                             mtb_thresh[...,0],
    #                             mtb_thresh[...,1]], axis = -1)

    # check channel axis order
    intensity_image = np.transpose(images[:,1:,...], (0, 2, 3, 4, 1))

    # del mtb_images
    # del mtb_thresh

    objects = btrack.utils.segmentation_to_objects(
        segmentation, intensity_image=intensity_image,
        properties = tuple(['area', 'mean_intensity']), 
        use_weighted_centroid=False,
    )

    objects_fn = os.path.join(label_dir, 'objects.h5')
    with btrack.io.HDF5FileHandler(objects_fn, 'w', obj_type='obj_type_1') as writer:
        writer.write_objects(objects)

    tracks_fn = os.path.join(label_dir, 'tracks.h5')
    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
    
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/btrack/particle_config_pnassign.json')
        tracker.max_search_radius = 50
        # tracker.tracking_updates = ["MOTION", "VISUAL"]
        # tracker.features = FEATURES
    
        # append the objects to be tracked
        tracker.append(objects)
    
        # set the tracking volume
        tracker.volume=((0, segmentation.shape[2]), (0, segmentation.shape[3]), (0, segmentation.shape[1]))
    
        # track them (in interactive mode)
        tracker.track(step_size=10)
    
        # generate hypotheses and run the global optimizer
        tracker.optimize()
    
        # get the tracks in a format for napari visualization
        data, properties, graph = tracker.to_napari()
        
        # store the tracks
        tracks = tracker.tracks
        
        # store the configuration
        cfg = tracker.configuration
        
        # export the track data 
        tracker.export(tracks_fn, obj_type="obj_type_1")
        

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/289 [00:00<?, ?it/s]

Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is depricated in v4.0.1+
Resizing is de

KeyboardInterrupt: 